In [1]:
import os   

In [2]:
%pwd

'd:\\end to end mental_health_prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\end to end mental_health_prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path
from typing import List, Dict, Any, Optional

In [6]:
# mlProject/entity/config_entity.py
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
# mlProject/config/configuration.py
from mlProject.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH, SCHEMA_FILE_PATH
from mlProject.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        # Force convert to Path object in case they are strings
        config_filepath = Path(config_filepath)
        params_filepath = Path(params_filepath)
        schema_filepath = Path(schema_filepath)

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [9]:
# mlProject/components/data_ingestion.py
import os
import urllib.request as request
import zipfile
from pathlib import Path
from mlProject import logger
from mlProject.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! Info: \n{headers}")
        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to: {unzip_path}")

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-07-20 20:45:16,095: INFO: common: yaml file: D:\end to end mental_health_prediction\config.yaml loaded successfully]
[2025-07-20 20:45:16,098: INFO: common: yaml file: D:\end to end mental_health_prediction\params.yaml loaded successfully]
[2025-07-20 20:45:16,099: INFO: common: yaml file: D:\end to end mental_health_prediction\schema.yaml loaded successfully]
[2025-07-20 20:45:16,099: INFO: common: created directory at: artifacts]
[2025-07-20 20:45:16,099: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-20 20:45:18,006: INFO: 3307092374: artifacts/data_ingestion/survey.zip downloaded! Info: 
Connection: close
Content-Length: 47226
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "f16fe3c88c22772a1f26f1fe8eb01d80693f7c860caa7ff40583a008e275e0e8"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mo